# Explanation
This version assumes factor analysis provides the factors.

References, docs
https://www.datacamp.com/community/tutorials/introduction-factor-analysis
https://factor-analyzer.readthedocs.io/en/latest/index.html

In [297]:
%%capture
!pip install factor_analyzer
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [298]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("io.excel.xls.writer", "xlwt")
df = pd.read_pickle('~/Downloads/responses_latest.pkl')
df = df[
    (df['survey_short_id'] == 15) & (df['has_valid_response'] == True)
    ]

df[['scale_score', 'analysis_group', 'choice']] = df[['scale_score', 'analysis_group', 'choice']].fillna(0).astype(int)
#df['question_text'] = df['question_text'].replace('<[^<]+?>', '', regex=True)

# A bit more cleanup
# Not doing audio for the moment
df = df[df.response_type != 'AU']
# Move CH and Scale Score to a new combined column
# https://stackoverflow.com/questions/29991862/copy-value-from-one-column-based-on-the-value-of-another-column
df['score'] = 0
df.loc[df['response_type'] == 'SC', 'score'] = df['scale_score']
df.loc[df['response_type'] == 'CH', 'score'] = df['choice']

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.


In [299]:
#Create the pivot
df = df.pivot(index='respondent_id',
              columns=['analysis_group', 'question_sort_order', 'question_text', 'response_type'],
              values='score')
df.sort_index(axis=1, level=['analysis_group', 'question_sort_order'], inplace=True)
df.dropna(inplace=True)
df = df.astype('int16')

# 1 Predict Impfstatus from Demographics
1;Ich bin schon länger geimpft

2;Ich habe mich vor kurzem impfen lassen

3;Ich bin nicht geimpft

4;Ich bin genesen und geimpft (oder lasse mich impfen)

5;Ich bin genesen und lasse mich nicht impfen

In [300]:
import numpy as np
Y_vals = df.xs(44, level='question_sort_order', axis=1, drop_level=False)
#Y_vals.columns = Y_vals.columns.droplevel(0).droplevel(2).to_flat_index()
X_vals = df.loc[:, (slice(0),(10, 20, 30, 40))]
X_vals.columns = X_vals.columns.droplevel(0).droplevel(2).to_flat_index()
X_vals.columns = [' '.join(map(str, x)) for x in X_vals.columns]
Y_vals = np.ravel(Y_vals)

In [301]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_vals, Y_vals, test_size=0.3, random_state=2)

In [302]:
#clf = RandomForestClassifier()
clf = LinearSVC(max_iter=2000)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.875

# 2 Predict Impfstatus from motivation questions

In [303]:
X_vals = df.drop(0, axis=1, level=0)
X_vals.columns = X_vals.columns.droplevel(0).droplevel(2).to_flat_index()
X_vals.columns = [' '.join(map(str, x)) for x in X_vals.columns]
X_train, X_test, Y_train, Y_test = train_test_split(X_vals, Y_vals, test_size=0.3, random_state=11)
clf = LinearSVC(max_iter=6000)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.8333333333333334

## Factor Analysis

In [304]:
#https://www.datacamp.com/community/tutorials/introduction-factor-analysis
#do the factor analysis
# cols = list(range(1, 15))

#don't need any without the right group. In our case, groups are 1 - 14
df_4fa = df.copy(deep=True)
df_4fa = df_4fa.drop(0, axis=1, level=0) #drops all without a group, ie keeps all groups 1 - 14
df_4fa.columns = df_4fa.columns.to_flat_index()
df_4fa

,"(1, 80, Ich genieße es, mich in komplexe Sachverhalte einzuarbeiten., SC)","(1, 220, Es gefällt mir, Dinge zu untersuchen., SC)","(1, 360, Ich liebe es, zu denken., SC)","(2, 60, Ich lege großen Wert darauf, dass andere mich sympathisch finden., SC)","(2, 200, Ich fühle mich unwohl, wenn ich Dinge mache, die andere schlecht finden., SC)","(2, 340, Ich vermeide es, einen schlechten Eindruck auf andere zu machen., SC)","(3, 90, Ich fühle mich wohl damit, wenn ich Entscheidungen für andere treffen muss., SC)","(3, 230, Ich fühle mich wohl in der Rolle des Anführers., SC)","(3, 370, Es gefällt mir, wenn ich die Verantwortung für wichtige Entscheidungen trage., SC)","(4, 180, Meine Stellung in der Gesellschaft ist mir sehr wichtig., SC)","(4, 320, Ich suche nach Situationen, in denen ich mein soziales Ansehen erhöhen kann., SC)","(4, 460, Ich freue mich über die Stellung in der Gesellschaft, die ich erreicht habe., SC)","(5, 110, Es ist mir sehr wichtig, sparsam mit meinen Vorräten umzugehen., SC)","(5, 250, Ich bemühe mich, mein Geld und Gut zu erhalten., SC)","(5, 390, Andere würden mich als preisbewusst bezeichnen., SC)","(6, 150, Ich komme allein gut klar im Leben., SC)","(6, 290, Ich komme am besten klar im Leben, wenn ich mich nur auf mich selbst verlasse., SC)","(6, 430, Es wäre schrecklich für mich, von anderen abhängig zu sein., SC)","(7, 130, Meine Freundschaften und Bekanntschaften bereichern mein Leben., SC)","(7, 270, Ich lerne gerne neue Leute kennen., SC)","(7, 410, Ich genieße Gespräche und Plaudereien., SC)","(8, 170, Ich achte auf Sitten und Gebräuche, auf die Regeln des Zusammenlebens., SC)","(8, 310, Ich bin stolz auf meine moralischen Überzeugungen., SC)","(8, 450, Es fällt mir schwer, von Moralvorstellungen abzuweichen., SC)","(9, 100, Ich finde es wichtig, mich um Notleidende zu kümmern., SC)","(9, 240, Ich möchte einen Beitrag leisten, damit unsere Welt lebenswerter wird., SC)","(9, 380, Ich finde es furchtbar, Not und Benachteiligung zu sehen., SC)","(10, 120, Ich genieße es, wenn meine Wohnung aufgeräumt ist., SC)","(10, 260, Wenn ich Durcheinander sehe, möchte ich sofort Ordnung machen., SC)","(10, 400, Ich liebe es, Dinge zu ordnen und zu strukturieren., SC)","(11, 160, Ich umgehe Situationen, die andere für gefährlich halten., SC)","(11, 300, Ich lege großen Wert auf Sicherheit., SC)","(11, 440, Ich fühle mich sehr unwohl, wenn ich eine Situation nicht einschätzen kann., SC)","(12, 140, Wenn mich jemand beleidigt, muss ich es ihm heimzahlen., SC)","(12, 280, Ich lasse es mir nicht gefallen, wenn mich jemand provoziert., SC)","(12, 420, Ich finde es wichtig, anderen deutlich zu zeigen, dass sie mich verletzt haben., SC)","(13, 50, Ab hier geht's um Deine Motivationen. Ich genieße es, Sport zu treiben., SC)","(13, 190, Ich fühle mich wohl, wenn ich regelmäßig körperlich aktiv sein kann., SC)","(13, 330, Ich habe ein ausgeprägtes Bewegungsbedürfnis., SC)","(14, 70, Ich fühle mich sehr wohl auf Familienfesten., SC)","(14, 210, Ich achte auf Möglichkeiten, meine Familie unterstützen zu können., SC)","(14, 350, Ich denke häufig darüber nach, ob es den Mitgliedern meiner Familie gut geht., SC)"
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
865,3,2,2,5,5,5,4,4,5,4,3,4,5,4,5,4,2,5,6,4,6,3,1,1,2,2,4,6,6,6,6,6,6,5,5,6,6,5,5,5,4,3
866,3,3,3,5,6,6,5,5,5,2,1,5,3,3,3,5,5,2,2,1,1,1,1,1,2,2,2,4,4,4,2,2,1,2,2,2,5,6,6,2,1,1
887,4,4,5,4,3,3,2,3,4,4,3,4,5,3,1,5,3,3,3,4,4,3,4,4,4,4,3,4,5,4,4,4,4,4,3,3,3,4,4,3,4,3
910,5,6,6,4,3,4,4,5,5,4,3,5,4,4,3,6,4,4,6,5,5,3,5,4,5,6,6,4,4,4,2,4,5,2,4,5,6,6,5,4,6,5
913,4,2,4,3,2,3,4,2,4,2,2,4,4,4,3,4,3,3,4,2,3,4,4,4,4,4,3,4,2,3,4,4,4,3,2,3,1,2,2,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1484,5,5,4,5,4,6,4,3,4,5,6,6,4,5,3,3,3,2,6,6,5,5,6,4,4,6,6,5,3,3,4,6,3,1,3,3,5,6,4,6,6,6
1533,6,3,6,6,6,6,2,3,5,2,3,5,6,6,6,6,4,6,6,6,6,5,6,4,6,6,6,6,6,6,3,4,6,1

In [305]:
#https://www.datacamp.com/community/tutorials/introduction-factor-analysis
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(df_4fa)
# p value < 0.05 means we reject the null hypothesis that all variances re the same- this is what we want
chi_square_value, p_value

(1967.3542693904228, 2.6876601427658465e-88)

In [306]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(df_4fa)
#https://www.statisticshowto.com/kaiser-meyer-olkin/
#https://en.wikipedia.org/wiki/Kaiser%E2%80%93Meyer%E2%80%93Olkin_test
# Should be above .6
kmo_model

0.6291768849909464

In [307]:
# Create factor analysis object and perform factor analysis

from factor_analyzer import FactorAnalyzer
fa = FactorAnalyzer()
fa.fit(df_4fa)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
v

array([ 7.40818225,  3.89474924,  2.91973505,  1.92613844,  1.50122695,
        1.37848964,  0.95427335,  0.83443799,  0.70710053,  0.63714829,
        0.59071044,  0.40588504,  0.25201372,  0.1831414 ,  0.12400575,
        0.0478161 ,  0.01307614, -0.04707681, -0.06603846, -0.09071639,
       -0.12162206, -0.20911109, -0.22175955, -0.25525837, -0.29074874,
       -0.3205277 , -0.33628237, -0.35657744, -0.38380541, -0.39235399,
       -0.45431028, -0.46002282, -0.46662323, -0.50085675, -0.51487018,
       -0.543726  , -0.58347089, -0.58804397, -0.60966964, -0.6523092 ,
       -0.68288086, -0.72158763])

In [308]:
num_factors = (v>0.95).sum()
num_factors

7

In [309]:
fa = FactorAnalyzer(num_factors, rotation="varimax")
fa.fit(df_4fa)

FactorAnalyzer(n_factors=7, rotation='varimax', rotation_kwargs={})

In [310]:
loadings = pd.DataFrame(fa.loadings_)
loadings.insert(0,'questions',df_4fa.columns)
loadings.to_excel('~/Downloads/loadings.xls')
loadings

,questions,0,1,2,3,4,5,6
0,"(1, 80, Ich genieße es, mich in komplexe Sachv...",0.572383,0.109408,0.041126,0.108823,0.073094,-0.204580,0.144492
1,"(1, 220, Es gefällt mir, Dinge zu untersuchen....",0.585243,0.328679,0.050842,0.064029,0.233661,-0.101038,0.041867
2,"(1, 360, Ich liebe es, zu denken., SC)",0.521354,0.439801,-0.007998,0.151195,0.220072,-0.174177,0.135054
3,"(2, 60, Ich lege großen Wert darauf, dass ande...",0.009775,-0.088882,0.155291,0.616389,0.126983,0.213634,0.118607
4,"(2, 200, Ich fühle mich unwohl, wenn ich Dinge...",-0.292434,-0.028034,-0.004635,0.263327,-0.115097,0.338977,-0.049768
5,"(2, 340, Ich vermeide es, einen schlechten Ein...",-0.114005,0.202930,0.038296,0.582663,0.103647,0.053361,0.255963
6,"(3, 90, Ich fühle mich wohl damit, wenn ich En...",0.517550,-0.226073,-0.023716,0.394384,0.012106,0.167911,-0.032317
7,"(3, 230, Ich fühle mich wohl in der Rolle des ...",0.644957,-0.039583,-0.142220,0.161361,0.215912,0.099654,-0.003246
8,"(3, 370, Es gefällt mir, wenn ich die Verantwo...",0.663957,0.049095,0.145073,0.022659,0.177783,0.041757,0.097041
9,"(4, 180, Meine Stellung in der Gesellschaft is...",0.128418,0.237260,0.004510,0.665733,0.050845,0.137469,-0.009312


In [311]:
# Get variance of each factors
# Row 3 is cumulative variance explained by the factors

pd.DataFrame(fa.get_factor_variance())

,0,1,2,3,4,5,6
0,4.115744,3.695346,3.580468,2.986006,2.824690,2.202883,2.073514
1,0.097994,0.087984,0.085249,0.071095,0.067255,0.052450,0.049369
2,0.097994,0.185978,0.271228,0.342323,0.409577,0.462027,0.511396


In [312]:
Y_vals = df.loc[:, (slice(None), 44)].to_numpy().flatten()
Y_vals

array([1, 1, 3, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 3, 4, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 5, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 3, 2, 1, 4, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int16)

## Y values explanation
1;Ich bin schon länger geimpft

2;Ich habe mich vor kurzem impfen lassen

3;Ich bin nicht geimpft

4;Ich bin genesen und geimpft (oder lasse mich impfen)

5;Ich bin genesen und lasse mich nicht impfen